---
title: "Client Report - [Finding Relationships in Baseball]"
subtitle: "Course DS 250"
author: "[MIGUEL SMITH]"
format:
  html:
    self-contained: true
    page-layout: full
    title-block-banner: true
    toc: true
    toc-depth: 3
    toc-location: body
    number-sections: false
    html-math-method: katex
    code-fold: true
    code-summary: "Show the code"
    code-overflow: wrap
    code-copy: hover
    code-tools:
        source: false
        toggle: true
        caption: See code
execute: 
  warning: false
    
---

In [ ]:
import pandas as pd 
import numpy as np
import sqlite3
from lets_plot import *

LetsPlot.setup_html(isolated_frame=True)

In [ ]:
# Learn morea about Code Cells: https://quarto.org/docs/reference/cells/cells-jupyter.html

# Include and execute your code here
sqlite_file = 'lahmansbaseballdb.sqlite'
# this file must be in the same location as your .qmd or .py file
con = sqlite3.connect(sqlite_file)

## QUESTION|TASK 1

__Write an SQL query to create a new dataframe about baseball players who attended BYU-Idaho. The new table should contain five columns: playerID, schoolID, salary, and the yearID/teamID associated with each salary. Order the table by salary (highest to lowest) and print out the table in your report.__  


In [ ]:
# Include and execute your code here
q = """SELECT p.playerID as 'Player',
            sc.schoolID as 'School',
            s.yearID as 'Year',
            s.salary AS 'Salary', 
            s.teamID AS 'Team'
            
            
       FROM collegeplaying sc
       
       JOIN people p ON p.playerID = sc.playerID 
       JOIN salaries s ON p.playerID = s.playerID
       
       WHERE sc.schoolID = 'idbyuid'
       GROUP BY p.playerID
       ORDER BY s.salary DESC

    """
results = pd.read_sql_query(q,con)
results

## QUESTION|TASK 2

__This three-part question requires you to calculate batting average (number of hits divided by the number of at-bats)__  
    a. Write an SQL query that provides playerID, yearID, and batting average for players with at least 1 at bat that year. Sort the table from highest batting average to lowest, and then by playerid alphabetically. Show the top 5 results in your report.  
    b. Use the same query as above, but only include players with at least 10 at bats that year. Print the top 5 results.  
    c. Now calculate the batting average for players over their entire careers (all years combined). Only include players with at least 100 at bats, and print the top 5 results.  

Upon writing these queries I have found that there is a significant difference in results when changing parameters of At Bats. It makes sense 
to limit which players we take for these averages because we get more useable numbers instead of 'perfect' averages.


In [ ]:
# Include and execute your code here
q = """ SELECT playerID AS 'Player',
               yearID AS 'Year',
               AB AS 'At Bat',
               H AS 'Hits',
               CAST(H as FLOAT) / AB AS 'Batting Average'

        FROM batting
        GROUP BY playerID
        HAVING AB >= 1 AND H>=1
        ORDER BY CAST(H as FLOAT) / AB DESC, playerID ASC
        LIMIT 5
        



"""
results = pd.read_sql_query(q,con)
results

In [ ]:
# Include and execute your code here
q = """ SELECT playerID AS 'Player',
               yearID AS 'Year',
               AB AS 'At Bat',
               H AS 'Hits',
               CAST(H as FLOAT) / AB AS 'Batting Average'

        FROM batting
        GROUP BY playerID
        HAVING AB >= 10 AND H>=1
        ORDER BY CAST(H as FLOAT) / AB DESC, playerID 
        LIMIT 5
        



"""
results = pd.read_sql_query(q,con)
results

In [ ]:
# Include and execute your code here
q = """ SELECT 
               playerID AS 'Player',
               SUM(AB) AS 'Career At-Bats',
               SUM(H) AS 'Career Hits',
               CAST(SUM(H) as FLOAT) / SUM(AB) AS 'Batting Average'

        FROM batting 
        GROUP BY playerID
        HAVING SUM(AB) > 100
        ORDER BY CAST(SUM(H) as FLOAT) / SUM(AB) DESC,  playerID ASC
        LIMIT 5
        
        
        
    

"""
results = pd.read_sql_query(q,con)
results

## QUESTION|TASK 3

__Pick any two baseball teams and compare them using a metric of your choice (average salary, home runs, number of wins, etc). Write an SQL query to get the data you need, then make a graph using Lets-Plot to visualize the comparison. What do you learn?__

I decided to gather data of Outs Pitched for two teams on polar opposites of the United States: The Boston Red Sox and 
the Los Angeles Dodgers. I wanted to see the differences year to year. To my surprise, the data is similar. Note the years 1918, 1981, and 1994 where both have similar dips. This puts forth the question of what happened to the MLB in those years.


In [ ]:
# Include and execute your code here
q =""" SELECT tf.franchName 'Team Name', 
       t.yearID as 'Year',
       t.W AS 'Wins',
       t.G as 'Games',
       CAST(t.W as FLOAT) / t.G AS 'Win%',
       t.IPOuts as 'Outs Pitched'
    FROM teams t 
    JOIN teamsfranchises tf on tf.franchID = t.franchID
    WHERE tf.franchName IN ('Los Angeles Dodgers', 'Boston Red Sox')
    ORDER BY t.yearID

 



"""
results = pd.read_sql_query(q,con)
results



plot = (ggplot(results)
 + geom_line(aes(x='Year', y='Outs Pitched', color='Team Name'), size=1)
 + geom_point(aes(x='Year', y='Outs Pitched', color='Team Name'), size=2)
 + ggtitle('Red Sox vs Dodgers Performance Over Time')
 + xlab('Year')
 + ylab('Outs Pitched')
 + theme_classic()+
    scale_x_continuous(format="d")
)

plot.show()


---

## STRETCH QUESTION|TASK 1

__Advanced Salary Distribution by Position (with Case Statement):__  

    * Write an SQL query that provides a summary table showing the average salary for players in each position (e.g., pitcher, catcher, outfielder) across all years. Include the following columns:

        * position
        * average_salary
        * total_players
        * highest_salary  

    * The highest_salary column should display the highest salary ever earned by a player in that position. If no player in that position has a recorded salary, display “N/A” for the highest salary.  

    * Additionally, create a new column called salary_category using a case statement:  

        * If the average salary is above $1 million, categorize it as “High Salary.”  
        * If the average salary is between $500,000 and $1 million, categorize it as “Medium Salary.”  
        * Otherwise, categorize it as “Low Salary.”  

    * Order the table by average salary in descending order.
    * Print the top 10 rows of this summary table.  

_type your results and analysis here_


In [ ]:
# Include and execute your code here


## STRETCH QUESTION|TASK 2

__Advanced Career Longevity and Performance (with Subqueries):__

    * Calculate the average career length (in years) for players who have played at least one game. Then, identify the top 10 players with the longest careers (based on the number of years they played). Include their:  

        * playerID
        * first_name
        * last_name
        * career_length

    * The career_length should be calculated as the difference between the maximum and minimum yearID for each player. 

MAX and MIN functions work very well in this context. Nick Altrock sounds like he had a lot of endurance.


In [ ]:
# Include and execute your code here
q = """
SELECT p.playerID,
       p.nameFirst as 'first_name',
       p.nameLast as 'last_name',
       MAX(CAST(a.yearID as INTEGER)) - MIN(CAST(a.yearID as INTEGER)) AS 'career_length'
FROM people p
JOIN appearances a ON p.playerID = a.playerID
GROUP BY p.playerID
ORDER BY career_length DESC
LIMIT 10

"""
results = pd.read_sql_query(q,con)
results

---
